In [36]:
import pandas as pd
import numpy as np
import os

In [37]:
import sklearn as sklearn
from datetime import *

In [38]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.feature_selection import RFE

In [39]:
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import Pipeline
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix, classification_report

In [40]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV
#from modeldb.sklearn_native.ModelDbSyncer import *
#from modeldb.sklearn_native import SyncableMetrics

In [41]:
Scoring  = pd.read_csv("ScoringDataProfiled.csv")
Training = pd.read_csv("CustomerChurnDataProfiled.csv")

In [43]:
Training.columns

Index(['ACCOUNT_BALANCE_LOW', 'AGE', 'AVG_FAMILY_SIZE', 'CHURN_Y_N',
       'Credit_card_limit', 'DEPOSIT_AMT', 'FIRST_NAME', 'GENDER',
       'INACTIVE_CUST_FLAG', 'INCOME_LEVEL', 'Interest_rate', 'LAST_NAME',
       'MARITAL_STATUS', 'NUM_DEPOSIT_TRX', 'NUM_OF_AVG_RESOLUTION',
       'NUM_OF_COMPLAINTS', 'NUM_OF_ENQUIRIES', 'NUM_OF_GOOD_RESOLUTION',
       'NUM_OF_POOR_RESOLUTION', 'NUM_OF_PROD_SUPRT', 'NUM_WDWL_TRX',
       'OCCUPATION', 'WDWL_AMT', '_id', 'create_date', 'customer_id',
       'product_id', 'product_type', 'sentiment_polarity', 'sentiment_score'],
      dtype='object')

#### Dropping the Columns based on the data from Data Profiling of Data Explorer

In [44]:
Training.drop([ 'FIRST_NAME','INACTIVE_CUST_FLAG','LAST_NAME','NUM_DEPOSIT_TRX', 'NUM_OF_AVG_RESOLUTION','NUM_OF_GOOD_RESOLUTION','NUM_OF_POOR_RESOLUTION','create_date','product_type'],axis=1,inplace=True)

In [45]:
Scoring.columns

Index(['ACCOUNT_BALANCE_LOW', 'AGE', 'AVG_FAMILY_SIZE', 'Credit_card_limit',
       'DEPOSIT_AMT', 'FIRST_NAME', 'GENDER', 'INACTIVE_CUST_FLAG',
       'INCOME_LEVEL', 'Interest_rate', 'LAST_NAME', 'MARITAL_STATUS',
       'NUM_DEPOSIT_TRX', 'NUM_OF_AVG_RESOLUTION', 'NUM_OF_COMPLAINTS',
       'NUM_OF_ENQUIRIES', 'NUM_OF_GOOD_RESOLUTION', 'NUM_OF_POOR_RESOLUTION',
       'NUM_OF_PROD_SUPRT', 'NUM_WDWL_TRX', 'OCCUPATION', 'WDWL_AMT', '_id',
       'create_date', 'customer_id', 'product_id', 'product_type',
       'sentiment_polarity', 'sentiment_score'],
      dtype='object')

In [46]:
Scoring.drop([ 'FIRST_NAME','INACTIVE_CUST_FLAG','LAST_NAME','NUM_DEPOSIT_TRX', 'NUM_OF_AVG_RESOLUTION','NUM_OF_GOOD_RESOLUTION','NUM_OF_POOR_RESOLUTION','create_date','product_type'],axis=1,inplace=True)

In [47]:
from sklearn.model_selection import train_test_split

In [51]:
print (Training.describe())

       ACCOUNT_BALANCE_LOW         AGE  AVG_FAMILY_SIZE  Credit_card_limit  \
count           647.000000  647.000000       647.000000         647.000000   
mean              0.347759   44.244204         3.905719      237055.641422   
std               0.476627   13.069558         1.450552      210703.215914   
min               0.000000   22.000000         2.000000       25000.000000   
25%               0.000000   33.000000         3.000000      100000.000000   
50%               0.000000   45.000000         4.000000      100000.000000   
75%               1.000000   56.000000         5.000000      300000.000000   
max               1.000000   67.000000         6.000000      800000.000000   

        DEPOSIT_AMT  Interest_rate  NUM_OF_COMPLAINTS  NUM_OF_ENQUIRIES  \
count    647.000000     647.000000         647.000000        647.000000   
mean   14000.344668      15.965997           0.234930          0.168470   
std    15902.066713       1.539597           0.435091          0.374573 

In [52]:
print (Training.dtypes)

ACCOUNT_BALANCE_LOW    float64
AGE                    float64
AVG_FAMILY_SIZE          int64
CHURN_Y_N               object
Credit_card_limit      float64
DEPOSIT_AMT            float64
GENDER                  object
INCOME_LEVEL            object
Interest_rate          float64
MARITAL_STATUS          object
NUM_OF_COMPLAINTS      float64
NUM_OF_ENQUIRIES       float64
NUM_OF_PROD_SUPRT      float64
NUM_WDWL_TRX           float64
OCCUPATION              object
WDWL_AMT               float64
_id                     object
customer_id              int64
product_id               int64
sentiment_polarity      object
sentiment_score        float64
dtype: object


In [53]:
Training['interest_rate_low'] = np.where((Training['Interest_rate'] <= 16.0),1,0)
Training['interest_rate_high'] = np.where((Training['Interest_rate'] > 16.0),1,0)
Training['credit_limit_low'] = np.where((Training['Credit_card_limit'] <= 75000.0),1,0)
Training['credit_limit_high'] = np.where((Training['Credit_card_limit'] > 75000.0),1,0)
Training['Nuclear_Family'] = np.where((Training['AVG_FAMILY_SIZE'] <= 4),1,0)
Training['Joint_Family'] = np.where((Training['AVG_FAMILY_SIZE'] > 4),1,0)

In [54]:
Training.head(5)

,ACCOUNT_BALANCE_LOW,AGE,AVG_FAMILY_SIZE,CHURN_Y_N,Credit_card_limit,DEPOSIT_AMT,GENDER,INCOME_LEVEL,Interest_rate,MARITAL_STATUS,...,customer_id,product_id,sentiment_polarity,sentiment_score,interest_rate_low,interest_rate_high,credit_limit_low,credit_limit_high,Nuclear_Family,Joint_Family
0,0.0,50.0,2,N,100000.0,20973.0,Female,Medium,16.0,Married,...,22278,3848,neutral,0.0,1,0,0,1,1,0
1,0.0,33.0,2,Y,50000.0,61346.0,Male,High,18.0,Married,...,22284,3354,neutral,0.0,0,1,1,0,1,0
2,0.0,66.0,6,N,400000.0,9636.0,Male,Low,12.0,Married,...,22264,1284,neutral,0.0,1,0,0,1,0,1
3,0.0,54.0,6,N,100000.0,24070.0,Male,High,16.0,Married,...,22268,1819,neutral,0.0,1,0,0,1,0,1
4,1.0,32.0,2,Y,300000.0,0.0,Male,Low,15.0,Married,...,22258,2547,negative,-1.0,1,0,0,1,1,0


In [55]:
Training['age_20_30'] = np.where((Training['AGE'] >= 20) & (Training['AGE'] <= 30),1,0)
Training['age_30_40'] = np.where((Training['AGE'] > 30) & (Training['AGE'] <= 40),1,0)
Training['age_40_50'] = np.where((Training['AGE'] > 40) & (Training['AGE'] <= 50),1,0)
Training['age_50_60'] = np.where((Training['AGE'] > 50) & (Training['AGE'] <= 60),1,0)
Training['age_60_70'] = np.where((Training['AGE'] > 60) & (Training['AGE'] <= 70),1,0)

In [56]:
X = Training.filter(['customer_id','GENDER','INCOME_LEVEL','MARITAL_STATUS','sentiment_polarity'],axis=1)
def dummy_df(df,todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x],prefix = x,dummy_na = False)
        df = df.drop(x,1)
        df = pd.concat([df,dummies],axis=1)
    return df

catg_list = ['GENDER','INCOME_LEVEL','MARITAL_STATUS','sentiment_polarity']
#for col_name in X.columns:
 #   if X[col_name].dtypes == 'object':
        #catg_list.append(col_name)

print(catg_list)
X = dummy_df(X,catg_list)
print (X)
X.columns

['GENDER', 'INCOME_LEVEL', 'MARITAL_STATUS', 'sentiment_polarity']
     customer_id  GENDER_Female  GENDER_Male  INCOME_LEVEL_High  \
0          22278              1            0                  0   
1          22284              0            1                  1   
2          22264              0            1                  0   
3          22268              0            1                  1   
4          22258              0            1                  0   
5          22262              1            0                  1   
6          22269              0            1                  0   
7          22281              1            0                  0   
8          22252              0            1                  0   
9          22248              1            0                  0   
10         22274              0            1                  1   
11         22256              0            1                  1   
12         22246              0            1                  

Index(['customer_id', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High',
       'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Married',
       'MARITAL_STATUS_Single', 'sentiment_polarity_negative',
       'sentiment_polarity_neutral', 'sentiment_polarity_positive'],
      dtype='object')

In [57]:
Training = pd.merge(Training,X,on='customer_id',how='left')
Training.columns

Index(['ACCOUNT_BALANCE_LOW', 'AGE', 'AVG_FAMILY_SIZE', 'CHURN_Y_N',
       'Credit_card_limit', 'DEPOSIT_AMT', 'GENDER', 'INCOME_LEVEL',
       'Interest_rate', 'MARITAL_STATUS', 'NUM_OF_COMPLAINTS',
       'NUM_OF_ENQUIRIES', 'NUM_OF_PROD_SUPRT', 'NUM_WDWL_TRX', 'OCCUPATION',
       'WDWL_AMT', '_id', 'customer_id', 'product_id', 'sentiment_polarity',
       'sentiment_score', 'interest_rate_low', 'interest_rate_high',
       'credit_limit_low', 'credit_limit_high', 'Nuclear_Family',
       'Joint_Family', 'age_20_30', 'age_30_40', 'age_40_50', 'age_50_60',
       'age_60_70', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High',
       'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Married',
       'MARITAL_STATUS_Single', 'sentiment_polarity_negative',
       'sentiment_polarity_neutral', 'sentiment_polarity_positive'],
      dtype='object')

In [58]:
df_main = Training.drop(['GENDER','AGE', 'AVG_FAMILY_SIZE', 'Credit_card_limit', 'DEPOSIT_AMT', 'INCOME_LEVEL', 'Interest_rate',
        'MARITAL_STATUS','OCCUPATION','sentiment_polarity','_id','WDWL_AMT','product_id','sentiment_score'],axis =1)

In [59]:
Y_x = df_main['CHURN_Y_N']
X_1 = df_main.drop(['customer_id','CHURN_Y_N'],axis=1)
from sklearn.feature_selection import chi2,f_classif
labels = Y_x.values
features = X_1.values
chi2_, pval = chi2(features, labels)
print(chi2_)
print(pval)
#Selecting the variables whose Pval is lesser than 0.3
feat_list = list(X_1.columns)

len(feat_list)
len(chi2_)

serv_var_imp_x = pd.DataFrame(
    {'Variable': feat_list,
     'Scores': chi2_.tolist(),
     'Pval': pval.tolist()                 
    })

[1.38791333e+00 1.73497991e+00 9.31240440e-01 2.23735357e+00
 4.10146027e-02 2.01991913e+01 7.18351522e+01 1.18779280e+02
 1.16964317e+01 2.03158118e+00 3.06312046e+00 3.80798400e+00
 2.23463630e+00 9.12761566e-01 1.25520191e+01 9.49547147e-01
 1.35650506e+00 1.26337188e+00 1.94434783e+01 1.73379050e+00
 9.57888768e+00 4.20671521e-01 3.06874481e+00 5.81208570e+00
 1.40896806e+00 1.75273760e-01]
[2.38757804e-01 1.87776164e-01 3.34540805e-01 1.34711364e-01
 8.39509738e-01 6.97824292e-06 2.33947260e-17 1.17056172e-27
 6.26200693e-04 1.54060254e-01 8.00877307e-02 5.10088102e-02
 1.34948375e-01 3.39382754e-01 3.95778424e-04 3.29834594e-01
 2.44144105e-01 2.61013804e-01 1.03621055e-05 1.87927539e-01
 1.96827618e-03 5.16602159e-01 7.98110728e-02 1.59164064e-02
 2.35227777e-01 6.75466759e-01]


In [60]:
sel_features = serv_var_imp_x[serv_var_imp_x.Pval <= 0.5]
features = sel_features['Variable'].tolist()
features.append('customer_id')
features.append('CHURN_Y_N')
print (features)

['ACCOUNT_BALANCE_LOW', 'NUM_OF_COMPLAINTS', 'NUM_OF_ENQUIRIES', 'NUM_OF_PROD_SUPRT', 'interest_rate_low', 'interest_rate_high', 'credit_limit_low', 'credit_limit_high', 'Nuclear_Family', 'Joint_Family', 'age_20_30', 'age_30_40', 'age_40_50', 'age_50_60', 'age_60_70', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High', 'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Single', 'sentiment_polarity_negative', 'sentiment_polarity_neutral', 'customer_id', 'CHURN_Y_N']


In [61]:
df = df_main[features]
print (df.columns)
print (df.shape)

Index(['ACCOUNT_BALANCE_LOW', 'NUM_OF_COMPLAINTS', 'NUM_OF_ENQUIRIES',
       'NUM_OF_PROD_SUPRT', 'interest_rate_low', 'interest_rate_high',
       'credit_limit_low', 'credit_limit_high', 'Nuclear_Family',
       'Joint_Family', 'age_20_30', 'age_30_40', 'age_40_50', 'age_50_60',
       'age_60_70', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High',
       'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Single',
       'sentiment_polarity_negative', 'sentiment_polarity_neutral',
       'customer_id', 'CHURN_Y_N'],
      dtype='object')
(647, 25)


In [63]:
df1 = df.copy()

In [64]:
df1 = df1.drop(['customer_id'],axis=1)

In [65]:
df1.columns

Index(['ACCOUNT_BALANCE_LOW', 'NUM_OF_COMPLAINTS', 'NUM_OF_ENQUIRIES',
       'NUM_OF_PROD_SUPRT', 'interest_rate_low', 'interest_rate_high',
       'credit_limit_low', 'credit_limit_high', 'Nuclear_Family',
       'Joint_Family', 'age_20_30', 'age_30_40', 'age_40_50', 'age_50_60',
       'age_60_70', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High',
       'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Single',
       'sentiment_polarity_negative', 'sentiment_polarity_neutral',
       'CHURN_Y_N'],
      dtype='object')

In [66]:
all_inputs = df1[['ACCOUNT_BALANCE_LOW', 'NUM_OF_COMPLAINTS', 'NUM_OF_ENQUIRIES',
       'NUM_OF_PROD_SUPRT', 'interest_rate_low', 'interest_rate_high',
       'credit_limit_low', 'credit_limit_high', 'Nuclear_Family',
       'Joint_Family', 'age_20_30', 'age_30_40', 'age_40_50', 'age_50_60',
       'age_60_70', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High',
       'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Single',
       'sentiment_polarity_negative', 'sentiment_polarity_neutral']].values
all_classes = df1['CHURN_Y_N'].values

In [67]:
# Creating a new project
'''

name = "Customer Churn Use Case"
author = "Test_user_Churn"
description = "Classification Technique"
syncer_obj = Syncer(
    NewOrExistingProject(name, author, description),
    NewOrExistingExperiment("customerChurnUseCase test user churn", "churnDesc test user churn"),
    NewExperimentRun("churn use case test user churn"))
'''

'\n\nname = "Customer Churn Use Case"\nauthor = "Test_user_Churn"\ndescription = "Classification Technique"\nsyncer_obj = Syncer(\n    NewOrExistingProject(name, author, description),\n    NewOrExistingExperiment("customerChurnUseCase test user churn", "churnDesc test user churn"),\n    NewExperimentRun("churn use case test user churn"))\n'

In [68]:
#syncer_obj.add_tag(all_inputs, "data to input into model")

In [69]:
# Make sure that you don't mix up the order of the entries
# all_inputs[5] inputs should correspond to the class in all_classes[5]
'''
(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = cross_validation.train_test_split_sync(
    all_inputs, all_classes, train_size=0.80, random_state=1)
'''
(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(
    all_inputs, all_classes, train_size=0.80, random_state=1)

/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [70]:
classifier = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
#syncer_obj.add_tag(classifier, "random forest")

In [72]:
feature_columns1 = df1.columns

In [73]:
#classifier.fit_sync(training_inputs, training_classes)
classifier.fit(training_inputs, training_classes)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [75]:
# Validate the classifier on the testing set using classification accuracy
# decision_tree_classifier.score(testing_inputs, testing_classes)

# NOTE: score is equivalent to sklearn.metrics.accuracy_score.
'''
SyncableMetrics.compute_metrics(
    classifier, accuracy_score, testing_classes,
    classifier.predict(testing_inputs), training_inputs, "", "")

# cross_val_score returns a list of the scores, which we can visualize
# to get a reasonable estimate of our classifier's performance
cv_scores = cross_validation.cross_val_score_sync(
    classifier, all_inputs, all_classes, cv=10)
    '''

'\nSyncableMetrics.compute_metrics(\n    classifier, accuracy_score, testing_classes,\n    classifier.predict(testing_inputs), training_inputs, "", "")\n\n# cross_val_score returns a list of the scores, which we can visualize\n# to get a reasonable estimate of our classifier\'s performance\ncv_scores = cross_validation.cross_val_score_sync(\n    classifier, all_inputs, all_classes, cv=10)\n    '

In [76]:
#syncer_obj.sync()

In [77]:
pipeline = PMMLPipeline([("classifier", classifier)])

In [78]:
#pickle pipeline
pipeline_pickle_file = "/home/ubuntu/data-science-workbench/model-deployer/deploy/customer_churn_pipeline_test.pkl.z"
joblib.dump(pipeline, pipeline_pickle_file, compress = 9)

['/home/ubuntu/data-science-workbench/model-deployer/deploy/customer_churn_pipeline_test.pkl.z']

In [79]:
pipeline.fit(df1[['ACCOUNT_BALANCE_LOW', 'NUM_OF_COMPLAINTS', 'NUM_OF_ENQUIRIES',
       'NUM_OF_PROD_SUPRT', 'interest_rate_low', 'interest_rate_high',
       'credit_limit_low', 'credit_limit_high', 'Nuclear_Family',
       'Joint_Family', 'age_20_30', 'age_30_40', 'age_40_50', 'age_50_60',
       'age_60_70', 'GENDER_Female', 'GENDER_Male', 'INCOME_LEVEL_High',
       'INCOME_LEVEL_Low', 'INCOME_LEVEL_Medium', 'MARITAL_STATUS_Single',
       'sentiment_polarity_negative', 'sentiment_polarity_neutral']],df1['CHURN_Y_N'] )

PMMLPipeline(steps=[('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [80]:
df1.head()

,ACCOUNT_BALANCE_LOW,NUM_OF_COMPLAINTS,NUM_OF_ENQUIRIES,NUM_OF_PROD_SUPRT,interest_rate_low,interest_rate_high,credit_limit_low,credit_limit_high,Nuclear_Family,Joint_Family,...,age_60_70,GENDER_Female,GENDER_Male,INCOME_LEVEL_High,INCOME_LEVEL_Low,INCOME_LEVEL_Medium,MARITAL_STATUS_Single,sentiment_polarity_negative,sentiment_polarity_neutral,CHURN_Y_N
0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,N
1,0.0,0.0,0.0,0.0,0,1,1,0,1,0,...,0,0,1,1,0,0,0,0,1,Y
2,0.0,0.0,0.0,0.0,1,0,0,1,0,1,...,1,0,1,0,1,0,0,0,1,N
3,0.0,0.0,0.0,0.0,1,0,0,1,0,1,...,0,0,1,1,0,0,0,0,1,N
4,1.0,0.0,0.0,1.0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,1,0,Y


In [128]:
pwd